# Not successfull, next try LBLRTM

This notebook is developed within [this chat-gpt conversation](https://chatgpt.com/share/690910be-7590-8007-8eee-ccb2d9d744c6).

The goal is to esteblish a environment that can run arts, with spectral lines and back ground.

# install Conda

In [16]:
All in this cell should be executed in the terminal
# fresh env (Python 3.12 is fine)
mamba create -n arts python=3.12 -c rttools -c conda-forge pyarts matplotlib numpy
conda activate arts

# (optional) make kernal available to jupyter lab
mamba install ipykernel
python -m ipykernel install --user --name arts

# point ARTs to its data, this should probably made permanent in "$CONDA_PREFIX/etc/conda/activate.d"
export ARTS_DATA_PATH="~/.cache/arts/arts-xml-data-2.6.16"

# one still needs absoption lines, which we can get through hapi, install it:
# Install hapi; don't use the conda version as it is out of date
python -m pip install --upgrade hitran-api

SyntaxError: invalid syntax (1555330158.py, line 1)

In [1]:
# sanity check
import pyarts, sys
print("pyarts:", pyarts.__version__)
from pyarts.workspace import Workspace
ws = Workspace()
print("ARTS core ready.")

pyarts: 2.6.16
ARTS core ready.


# Get catalogs & set paths (one line)

In [2]:
from pyarts.cat import download
download.retrieve()

# Quick smoke test (creates a one-point absorption coefficient)

In [4]:
import os, pathlib
arts_data = os.path.expanduser("~/.cache/arts/arts-xml-data-2.6.16")  # expand "~"
assert pathlib.Path(arts_data).is_dir(), f"Path not found: {arts_data}"
os.environ["ARTS_DATA_PATH"] = arts_data

from pyarts.workspace import Workspace
ws = Workspace()
print("ARTS_DATA_PATH (kernel):", os.environ.get("ARTS_DATA_PATH"))


ARTS_DATA_PATH (kernel): /home/hagen/.cache/arts/arts-xml-data-2.6.16


In [5]:
# Frequency grid: 910–970 nm (ascending Hz)
c = 299_792_458.0
wl_nm = np.linspace(970.0, 910.0, 241)              # nm, high→low so freq ends up ascending
f_grid = c / (wl_nm * 1e-9)                         # Hz
ws.f_grid = f_grid

# Choose H2O + MT_CKD (v4.0.0 tags shown here) — continuum tags can be combined with line tags
ws.abs_speciesSet(species=[
    "H2O",
    "H2O-SelfContCKDMT400",
    "H2O-ForeignContCKDMT400",
])

## Before we continue we need some linebyline data. Get it from HiTran with hapi

In [1]:
# HAPI: download HITRAN H2O lines for 910–970 nm (≈ 10 309–10 989 cm^-1)

from hapi import *       # you already installed:  python -m pip install hapi
import os
from pathlib import Path

# --- user settings ---
out_dir = Path("./hitran_db")             # local HAPI DB folder
par_out = Path("./H2O_910_970.par")       # optional export to 160-char .par
# If you have a HITRAN login, set these in your environment for bigger queries:
# os.environ["HITRAN_USER"] = "your_email"
# os.environ["HITRAN_PASS"] = "your_password"

# --- setup HAPI local DB and (optionally) authenticate ---
out_dir.mkdir(parents=True, exist_ok=True)
db_begin(str(out_dir))                                     # initialize local DB
if os.getenv("HITRAN_USER") and os.getenv("HITRAN_PASS"):
    setAccess(os.getenv("HITRAN_USER"), os.getenv("HITRAN_PASS"))  # optional

# --- convert wavelength [nm] to wavenumber [cm^-1] ---
wl_min_nm, wl_max_nm = 910.0, 970.0
nu_max = 1.0 / (wl_min_nm * 1e-7)      # cm^-1  (shorter λ -> higher ν)
nu_min = 1.0 / (wl_max_nm * 1e-7)      # cm^-1
print(f"Requesting ν ∈ [{nu_min:.2f}, {nu_max:.2f}] cm^-1")

# --- fetch H2O lines (molecule #1) for main isotopologues (local ids 1–5) ---
table = "H2O_910_970"
for local_iso_id in (1, 2, 3, 4, 5):   # 1H2-16O, 1H2-18O, 1H2-17O, HDO, D2O
    fetch(table, 1, local_iso_id, nu_min, nu_max)  # (name, molec_id=1, iso_id, νmin, νmax)

# quick sanity checks
describeTable(table)
print("Lines in range:", len(getColumn(table, "nu")))

# --- optional: export a HITRAN '160-char' .par file for ARTS ---
try:
    # Many HAPI builds support exporting the canonical 160-char record via ParameterGroups
    select(table, File=str(par_out), ParameterGroups=("160-char",))
    print("Wrote HITRAN 160-char file:", par_out.resolve())
except Exception as e:
    print("Could not export 160-char .par via HAPI on this build:", e)
    # Fallback: write a plain-text CSV-like export with standard columns
    select(
        table,
        File=str(par_out.with_suffix(".txt")),
        ParameterNames=("molec_id","local_iso_id","nu","sw","A","gamma_air",
                        "gamma_self","n_air","delta_air","elow","gpp","iup","ilow","quanta")
    )
    print("Wrote text export:", par_out.with_suffix(".txt").resolve())


HAPI version: 1.2.2.4
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2021 are now available in HAPI code

           MIT license: Copyright 2021 HITRAN team, see more at http://hitran.org. 

           If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005

           ATTENTION: This is the core version of the HITRAN Application Programming Interface.
                      For more efficient implementation of the absorption coefficient routine, 
                      as well as for new profiles, parameters and other functional,
      

/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/site-packages/hapi/hapi.py:882: SyntaxWarning: invalid escape sequence '\%'
  FORMAT_PYTHON_REGEX = '^\%(\d*)(\.(\d*))?([edfsEDFS])$'
/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/site-packages/hapi/hapi.py:971: SyntaxWarning: invalid escape sequence '\%'
  regex = '^\%([0-9]+)\.?[0-9]*([dfs])$' #
/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/site-packages/hapi/hapi.py:1156: SyntaxWarning: invalid escape sequence '\d'
  res = re.search('(\d\.\d\d\d)\-(\d\d\d)',line[start:end])
/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/site-packages/hapi/hapi.py:1310: SyntaxWarning: invalid escape sequence '\.'
  matchObject = re.search('(.+)\.header$',file_name)
/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/site-packages/hapi/hapi.py:1326: SyntaxWarning: invalid escape sequence '\.'
  fname,fext = re.search('(.+)\.(\w+)',file_name).groups()
/home/hagen/programms/anaconda3/envs/arts/lib/python3.12/si

Exception: Cannot connect to http://hitran.org. Try again or edit GLOBAL_HOST variable.

In [14]:
# --- Step 3: Load HITRAN lines and convert to per-species ---

# 1) Frequency window (Hz) — using your existing f_grid
fmin = float(min(ws.f_grid.value))
fmax = float(max(ws.f_grid.value))

# 2) Read HITRAN lines directly into ws.abs_lines
#    (This is the ARTS/pyarts method name; no "abs_lines..." prefix.)
hitran_par = "/absolute/path/to/HITRAN2020/H2O.par"  # update this
ws.ReadHITRAN(
    filenames=[hitran_par],  # list is accepted
    fmin=fmin,
    fmax=fmax,
)

# 3) Convert the flat line list to the per-species structure required by the clear-sky agenda
ws.abs_lines_per_speciesCreateFromLines()

# Quick sanity check
print("n(abs_lines) =", len(ws.abs_lines.value))
print("n(abs_lines_per_species) =", len(ws.abs_lines_per_species.value))


TypeError: ReadHITRAN(): incompatible function arguments. The following argument types are supported:
    1. (self: pyarts.arts._Workspace, abs_lines: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.ArrayOfAbsorptionLines]] = self.abs_lines, filename: Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String], fmin: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.Numeric]] = -1e99, fmax: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.Numeric]] = 1e99, globalquantumnumbers: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String]] = "DEFAULT_GLOBAL", localquantumnumbers: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String]] = "DEFAULT_LOCAL", hitran_type: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String]] = "Online", normalization_option: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String]] = "None", mirroring_option: Optional[Union[pyarts.arts.WorkspaceVariable, pyarts.arts.String]] = "None", population_op

In [13]:
# --- Step 3 (replace your two calls with this single call) ---
hitran_par = "/absolute/path/to/HITRAN2020/H2O.par"  # update path

# Use the “catalogues” loader; HITRAN04 format parser handles HITRAN 2004+ .par files,
# which covers modern HITRAN releases including 2020.
ws.abs_lines_per_speciesReadFromCatalogues(
    filenames=[hitran_par],
    formats=["HITRAN04"],   # allowed formats listed in the docs include HITRAN96 and HITRAN04
    fmin=[float(min(ws.f_grid.value))],
    fmax=[float(max(ws.f_grid.value))],
)
# NOTE: Do *not* call abs_lines_per_speciesCreateFromLines() after this;
# this method already returns abs_lines_per_species directly.


AttributeError: 'Workspace' object has no attribute 'abs_lines_per_speciesReadFromCatalogues'

In [10]:
min(ws.f_grid.value)

np.float64(309064389690721.6)

In [12]:
# Narrow the line import to the band of interest (use your f_grid limits)
fmin = float(min(ws.f_grid.value))
fmax = float(max(ws.f_grid.value))

# Read HITRAN lines, then convert to per-species internal structure
ws.abs_linesReadFromHitran(filename="/path/to/HITRAN2020/H2O.par",
                           fmin=fmin, fmax=fmax)
ws.abs_lines_per_speciesCreateFromLines()


AttributeError: 'Workspace' object has no attribute 'abs_linesReadFromHitran'

In [10]:
min(ws.f_grid.value)

np.float64(309064389690721.6)

In [11]:
max(ws.f_grid.value)

np.float64(329442261538461.5)

In [1]:
import os, numpy as np
from pyarts.workspace import Workspace

ws = Workspace()
print("ARTS_DATA_PATH =", os.environ.get("ARTS_DATA_PATH"))

# Frequency grid: 910–970 nm (ascending Hz)
c = 299_792_458.0
wl_nm = np.linspace(970.0, 910.0, 241)              # nm, high→low so freq ends up ascending
f_grid = c / (wl_nm * 1e-9)                         # Hz
ws.f_grid = f_grid

# Choose H2O + MT_CKD (v4.0.0 tags shown here) — continuum tags can be combined with line tags
ws.abs_speciesSet(species=[
    "H2O",
    "H2O-SelfContCKDMT400",
    "H2O-ForeignContCKDMT400",
])


ARTS_DATA_PATH = None


In [2]:
os.environ.get("ARTS_DATA_PATH")

In [7]:
import numpy as np
from pyarts.workspace import Workspace
from pyarts.cat import download as catdl

# Ensure ARTS catalog + predefs (incl. MT_CKD) are available locally
catdl.retrieve(verbose=True)  # downloads arts-cat-data + arts-xml-data if missing :contentReference[oaicite:1]{index=1}

ws = Workspace()

# Species: group H2O lines + MT_CKD self/foreign continuum in one tag group
ws.abs_speciesSet(species=[
    "H2O, H2O-SelfContCKDMT400, H2O-ForeignContCKDMT400"
])

# Frequency window 910–970 nm
c = 299_792_458.0
fmin = c / (970e-9)          # Hz
fmax = c / (910e-9)          # Hz
ws.f_grid = np.linspace(fmin, fmax, 6001)

# Read HITRAN 2020 H2O lines (plain .par uses the "HITRAN04" format tag)
hitran_par = "/path/to/HITRAN2020/H2O.par"   # <-- set this
ws.abs_lines_per_speciesReadFromCatalogues(
    filenames=[hitran_par],
    formats=["HITRAN04"],                    # HITRAN .par reader
    fmin=[fmin], fmax=[fmax],
)  # creates abs_lines_per_species directly; supersedes the older two-step API. :contentReference[oaicite:2]{index=2}

# Toy state (1 level) — adapt to your profile as needed
ws.p_grid = np.array([1013e2])          # Pa
ws.abs_p  = np.array([1013e2])          # Pa
ws.abs_t  = np.array([288.0])           # K
ws.abs_vmrs = [np.array([0.01])]        # H2O VMR ~1%

# Build agendas and compute absorption
ws.propmat_clearsky_agendaAuto()
ws.abs_coefCalc()

# Quick sanity checks
print("n tag-groups:", len(ws.abs_lines_per_species.value))
print("H2O lines loaded:", len(ws.abs_lines_per_species.value[0]))
print("abs coef shape:", ws.abs_coef.value.shape)   # (nfreq, npressure, stokes_dim)


Skipping download, data already exists at /home/hagen/.cache/arts/arts-xml-data-2.6.16
Skipping download, data already exists at /home/hagen/.cache/arts/arts-cat-data-2.6.16


AttributeError: 'Workspace' object has no attribute 'abs_lines_per_speciesReadFromCatalogues'

In [ ]:
ws.abs_lin

In [6]:
import numpy as np, os
from pyarts.workspace import Workspace
from pyarts.cat import download as catdl

# 1) Make sure catalogs are available locally (arts-cat-data, arts-xml-data)
catdl.retrieve(verbose=True)   # downloads/caches if missing. :contentReference[oaicite:0]{index=0}

ws = Workspace()

# 2) Define species: H2O lines + MT_CKD continuum (choose the MT_CKD flavor you want)
ws.abs_speciesSet(species=[
    "H2O",
    "H2O-SelfContCKDMT400",
    "H2O-ForeignContCKDMT400",
])

# 3) Load HITRAN2020 lines for H2O in the requested spectral window
c = 299_792_458.0
fmin = c / (970e-9)   # Hz
fmax = c / (910e-9)   # Hz

# Option A: from a local HITRAN 2020 .par file (recommended: narrow fmin/fmax)
ws.abs_linesReadFromHitran(filename="/path/to/HITRAN2020/H2O.par", fmin=fmin, fmax=fmax)
ws.abs_lines_per_speciesCreateFromLines()  # turn the read lines into per-species structure
# (These are standard ARTS methods for reading HITRAN and creating per-species lines.) :contentReference[oaicite:1]{index=1}

# --- OR, Option B: generic interface if you prefer (same effect) ---
# ws.abs_lines_per_speciesReadFromCatalogues(
#     filenames=["/path/to/HITRAN2020/H2O.par"],
#     formats=["HITRAN04"],            # HITRAN .par reader
#     fmin=[fmin], fmax=[fmax],
# )  # :contentReference[oaicite:2]{index=2}

# 4) Frequency grid and state (toy example VMR/temperature/pressure)
ws.f_grid = np.linspace(fmin, fmax, 6001)
ws.p_grid = np.array([1013e2])          # Pa
ws.t_surface = 288.0
ws.abs_t = np.array([288.0])            # K at the single p level
ws.abs_p = np.array([1013e2])           # Pa
ws.abs_vmrs = [np.array([0.01])]        # H2O VMR ~1%

# 5) Let ARTS assemble agendas & compute absorption
ws.propmat_clearsky_agendaAuto()
ws.abs_coefCalc()

# Quick sanity check: how many H2O lines were picked up?
print("lines per tag:", [len(x) for x in ws.abs_lines_per_species.value])


Skipping download, data already exists at /home/hagen/.cache/arts/arts-xml-data-2.6.16
Skipping download, data already exists at /home/hagen/.cache/arts/arts-cat-data-2.6.16


AttributeError: 'Workspace' object has no attribute 'abs_linesReadFromHitran'